* [Grokking coding interview](https://www.educative.io/courses/grokking-coding-interview), [LC Company-wise Questions](https://github.com/hxu296/leetcode-company-wise-problems-2022?tab=readme-ov-file#airbnb)

In [ ]:
# @title import before you begin
from functools import lru_cache, partial, reduce
from itertools import accumulate, islice, chain, count, pairwise, product, tee
from collections import namedtuple, Counter, defaultdict, deque
from dataclasses import dataclass
from enum import Enum, IntFlag
from heapq import heappush, heappop, heapify
from typing import Optional, Union
from math import *
from operator import itemgetter
from sys import maxsize as INF  # infinity: 2**63-1.
import bisect
import numpy as np
import operator
import random
import re

class peekable:  # just like more_itertools.peekable
  def __init__(self, iterable, peeked=None):
    self.iterator, self.peeked = iter(iterable), peeked
  def __iter__(self):
    return self
  def __next__(self):
    if self.peeked is None:
      return next(self.iterator)
    peeked, self.peeked = self.peeked, None
    return peeked
  def peek(self, default=None):
    if self.peeked is None:
      try:
        self.peeked = next(self.iterator)
      except StopIteration:
        return default
    return self.peeked

  @classmethod
  def flatten(cls, iterable):
    def iterate():
      s = [iter(iterable)]
      while s:
        it = s.pop()
        while (e := next(it, None)) is not None:
          if isinstance(e, (list, tuple)):
            s.append(it)
            it = iter(e)
          else:
            yield e

    return cls(iterate())

Cut = namedtuple("Cut", "start stop", defaults=[0, 0])
Cut.off = classmethod(lambda cls, stop: cls(start=0, stop=stop))
Cut.__abs__ = lambda self: abs(self.stop - self.start)
Cut.__int__ = lambda self: self.stop - self.start
Cut.__call__ = lambda self, seq: seq[self.start:self.stop]

def minima_it(iterable, key=lambda e: e, inverse=False):
  it = tee(iterable, 2)
  key_it = tee(map(key, it[1]), 2)
  min_max = max if inverse else min
  min_max_key = min_max(key_it[1], default=None)
  yield from (e for key, e in zip(key_it[0], it[0]) if key == min_max_key)

def minima(iterable, key=lambda e: e, inverse=False, iterate=False):
  iterable = minima_it(iterable, key, inverse)
  return iterable if iterate else tuple(iterable)

def maxima(iterable, key=lambda e: e):
  it = tee(iterable, 2)
  key_it = tee(map(key, it[1]), 2)
  max_key = max(key_it[1], default=None)
  return tuple(e for key, e in zip(key_it[0], it[0]) if key == max_key)

maxima = partial(minima, inverse=True)
maxima_it = partial(minima_it, inverse=True)
recap = lambda kv, k, v: (kv[0] + k, kv[1] + v)

def pp(obj):
  from collections.abc import Mapping
  _prune = lambda v: (
      dict.fromkeys(v, "…") if isinstance(v, Mapping) else  #
      "…" if isinstance(v, obj.__class__) else v)

  m = len(values := list(_prune(v) for (k, v) in vars(obj).items() if k[0] != "_"))
  n = next((i for i, e in enumerate(reversed(values)) if e is not None), m)
  return f"{obj.__class__.__name__}({repr(values[:m-n])[1:-1]})".replace("'…'", "⋯")

def renumerate(it, stop=None):
  return (L := list(it)).reverse() or zip(count(stop or len(L), -1), L)

def dedupe(iterable):
  seen = set()
  for e in iterable:
    if e not in seen:
      seen.add(e)
      yield e

class DNode:
  def __init__(self, data, prev=None, next_=None):
    self.data, self.prev, self.next = data, prev, next_

def snode_from_values(cls, *values):
  return reduce(lambda acc, e: cls(e, acc), reversed(values), None)

@dataclass
class SNode:
  data: int = None
  next: "SNode" = None
  #
  def __iter__(self):
    L = self
    while L:
      yield L
      L = L.next

SNode.from_values = classmethod(snode_from_values)

def btree_from_values(cls, values, start=0, stop=None):
  if stop is None:
    stop = len(values)
  if stop > start:
    mid = (start+stop-1)//2  # mid becomes start, when stop-start: 2.
    L = btree_from_values(cls, values, start, mid)
    R = btree_from_values(cls, values, mid+1, stop)
    return cls(values[mid], L, R, None)

class BTree:
  def __init__(self, data=None, left=None, right=None, key=None):
    lr = (c if isinstance(c, (BTree, type(None))) else BTree(c) for c in (left, right))
    self.data, self.left, self.right = data, *lr
    self.key = data if key is None else key
    self._size = sum(bool(c) and c._size for c in (self.left, self.right))+1
    self._x = None

def iterate(tree):  # returns an iterator.
  stack = []
  while tree or stack:
    if tree:
      stack.append(tree)
      tree = tree.left
    else:
      yield (tree := stack.pop())
      tree = tree.right

BTree.__repr__ = pp  # yapf:disable
BTree.from_values = classmethod(btree_from_values)
T = lambda seq: isinstance(seq, list) and list(zip(*seq)) or tuple(zip(*seq))

* LC 2 https://leetcode.com/problems/add-two-numbers/, 0 given 0 and 0.
* LC 3 https://leetcode.com/problems/longest-substring-without-repeating-characters
* LC 4 https://leetcode.com/problems/median-of-two-sorted-arrays -- quick find the n-th smallest (or largest).
* LC 5 https://leetcode.com/problems/longest-palindromic-substring/, "bab" given "babad".
* LC 7 https://leetcode.com/problems/reverse-integer/, 21 for -120
* LC 10, 44 [Match Regular Expression](https://github.com/doocs/leetcode/blob/main/solution/0000-0099/0010.Regular%20Expression%20Matching/README_EN.md) (., *), [Match Wildcard](https://leetcode.com/problems/wildcard-matching/) (?, *)
* LC 11 https://leetcode.com/problems/container-with-most-water/, 1 given [1, 2]
* LC 15 3 sum https://leetcode.com/problems/3sum/description/
* LC 16 3 sum closest https://leetcode.com/problems/3sum-closest/description/
* LC 17 https://leetcode.com/problems/,letter-combinations-of-a-phone-number/, ["ad", "ae", ..., "cf"] given 23.
* LC 20 https://leetcode.com/problems/valid-parentheses, true for "([]{})"
* LC 21 https://leetcode.com/problems/merge-two-sorted-lists/, 1->2->4, 1->3->4
* LC 25 [Reverse Nodes in k-Group](https://leetcode.com/problems/reverse-nodes-in-k-group) -- tail_save = node; next_save = node.next
* LC 39 Combination Sum, `make_amount(denominations, k)`
* LC 41 https://leetcode.com/problems/first-missing-positive, 2 given [3, -1, 1].
* LC 42 Trapping Rain Water, 9 given [4, 2, 0, 3, 2, 5].
* LC 68 https://leetcode.com/problems/text-justification
* LC 76 https://leetcode.com/problems/minimum-window-substring
* LC 99 Recover BST
* LC 103, LC 987 Binary Tree Zigzag Level-order Traversal, Vertical order traversal
* LC 127 https://leetcode.com/problems/word-ladder/, ["hit", "hot", "dot", "dog", "cog"]
* LC 151 Reverse Words in a String
* LC 210 [Course Schedule II](https://github.com/doocs/leetcode/tree/main/solution/0200-0299/0210.Course%20Schedule%20II) -- prerequisites
* LC 224 https://leetcode.com/problems/basic-calculator/ -- produce and evaluate postfix.
* LC 240 [Search a 2D Matrix II](https://leetcode.com/problems/search-a-2d-matrix-ii/) -- find_in_2d_matrix(grid, query, rows, cols)
* LC 253 [Meeting Rooms II](https://github.com/doocs/leetcode/blob/main/solution/0200-0299/0253.Meeting%20Rooms%20II/README_EN.md) --   heapify(pq := reduce(lambda acc, intvl: acc + [(intvl[0], 1), (intvl[1], -1)], intervals, []))
* LC 273 https://leetcode.com/problems/integer-to-english-words/
* LC 211 https://leetcode.com/problems/design-add-and-search-words-data-structure
* LC 212 https://leetcode.com/problems/word-search-ii, [Boggle game](https://leetcode.com/discuss/interview-question/1036815/amazon-sde2-onsite-implement-boggle-game-trie-dfs-scalability-follow-up-question) -- backtrack and trie
* LC 236 https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-tree
* LC 251 Flatten 2D vector, LC 341 Flatten nested iterator
* LC 269 [Alien Dictionary](https://github.com/doocs/leetcode/tree/main/solution/0200-0299/0269.Alien%20Dictionary), topological sort in a stack ["x", "y"], given ["x", "y"].
* LC 285 [Inorder Successor in BST](https://github.com/doocs/leetcode/tree/main/solution/0200-0299/0285.Inorder%20Successor%20in%20BST) -- bisect_greater(tree, query).
* LC 295 [Median from Stream](https://leetcode.com/problems/find-median-from-data-stream/) -- max_q (negated), min_q.
* LC 300 https://leetcode.com/problems/longest-increasing-subsequence, 4 given [10,9,2,5,3,7,101,18]
* LC 322 https://leetcode.com/problems/coin-change, make change 7 out of denominations (5, 1).
* LC 324 Wiggle Sort II
* LC 336 [Palindrome Pairs](https://github.com/doocs/leetcode/tree/main/solution/0300-0399/0336.Palindrome%20Pairs)
* LC 359 [Rate-Limit Log Messages](https://github.com/doocs/leetcode/blob/main/solution/0300-0399/0359.Logger%20Rate%20Limiter/README_EN.md), limiter[message] = `time_us +10 *1000 *1000`
* LC 528 Random Pick with Weights
* LC 751 IP to CIDR
* LC 755 [Pour Water](https://github.com/doocs/leetcode/blob/main/solution/0700-0799/0755.Pour%20Water/README_EN.md), `pour_water(heights, valume)`.
* LC 756 Pyramid transition matrix
* LC 759 Employee Free Time
* LC 773 Sliding Puzzle -- bfs: `game, moves = q.popleft()`.
* LC 787 Flights in k Stops -- Dijkstra over graph G' with vertex (v, stops).
* LC 815 [Minimal Bus Rides](https://leetcode.com/problems/bus-routes), `bus_route_bfs(routes, source, sink)`
* LC 829 [Consecutive numbers sum](https://github.com/doocs/leetcode/blob/main/solution/0800-0899/0829.Consecutive%20Numbers%20Sum/README_EN.md)
* LC 843 https://leetcode.com/problems/guess-the-word
* LC 1275 https://leetcode.com/problems/find-winner-on-a-tic-tac-toe-game/
* LC 1386 [Cinema Seat Allocation](https://github.com/doocs/leetcode/blob/main/solution/1300-1399/1386.Cinema%20Seat%20Allocation/README_EN.md) -- (0b0111100000, 0b0000011110, 0b0001111000)
* LC 1404. [Number of Steps to Reduce a Number in Binary Representation to One](https://github.com/doocs/leetcode/blob/main/solution/1400-1499/1404.Number%20of%20Steps%20to%20Reduce%20a%20Number%20in%20Binary%20Representation%20to%20One/README_EN.md)
* LC 1405 [Longest Happy String](https://github.com/doocs/leetcode/blob/main/solution/1400-1499/1405.Longest%20Happy%20String/README_EN.md)
* LC 1647 [Minimum Deletions to Make Char Frequencies Unique](https://github.com/doocs/leetcode/blob/main/solution/1600-1699/1647.Minimum%20Deletions%20to%20Make%20Character%20Frequencies%20Unique/README_EN.md)
* LC 1653 [Minimum Deletions to Make String Balanced](https://github.com/doocs/leetcode/blob/main/solution/1600-1699/1653.Minimum%20Deletions%20to%20Make%20String%20Balanced/README_EN.md)
* LC 1775 [Equal Sum Arrays With Minimum Number of Operators](https://leetcode.com/problems/equal-sum-arrays-with-minimum-number-of-operations/description/)
* LC 2035 Partition Array Into Two Arrays to Minimize Sum Difference
* LC 2259 https://leetcode.com/problems/remove-digit-from-number-to-maximize-result

In [ ]:
ages = range(20, 121)
quantiles = np.percentile(ages, range(0, 101))
histogram, bin_edges = np.histogram(ages, bins=100, density=False)
cdf = (cumsum := np.cumsum(histogram)) / cumsum[-1]

**!! LP subsequence, LC substring, LC subsequence, Minimal # partitions, Longest increasing subsequence** Find the minimum # of partitions to make all partitions palindromic?

In [ ]:
def lcsubstring(L, R):  # LCS, longest common substring, https://leetcode.com/problems/maximum-length-of-repeated-subarray/ yapf:disable
  def lcsuff(m, n):
    if m>0 and n>0 and L[m-1] == R[n-1]:
      start, stop = lcsuff(m-1, n-1)
      return (start, stop+1) if stop > start else (m-1, m)
    return (0, 0)

  cases = (lcsuff(m, n) for m in range(len(L)+1) for n in range(len(R)+1))
  return tuple(L[slice(*c)] for c in maxima(cases, key=lambda s: s[1] - s[0]))

def lcsubseq(L, R):  # LCSubseq, longest common subsequence, https://wikipedia.org/wiki/longest_common_subsequence yapf:disable
  def prog(m, n):  # m chars and n chars to process.
    if m==0 or n==0:
      yield ""
    elif L[m-1] == R[n-1]:
      yield from (subseq + L[m-1] for subseq in prog(m-1, n-1))
    else:
      yield from (maxima(chain(prog(m-1, n), prog(m, n-1)), key=len))

  return dedupe(prog(len(L), len(R)))

def lisubsequence(L):  # LIS, longest increasing subsequence.
  def lisubseq(i):  # longest increasing subsequence ending at index i.
    cases = chain([(L[i],)], (
        subseq + (L[i],)  #
        for j in range(i)
        for subseq in lisubseq(j)
        if subseq[-1] <= L[i]))  # 0 <= j < i
    return maxima(cases, key=len)

  return maxima((subseq for i in range(len(L)) for subseq in lisubseq(i)), key=len)

def lisubseq_bottom_up(L, debug=False):
  memos = []
  for i, e in enumerate(L):
    for j, subseq in reversed(list(enumerate(memos))):
      if subseq[-1] <= L[i]:
        break
    else:
      j = -1
    subseq = memos[j] + [L[i]] if j != -1 else [L[i]]
    if j < len(memos)-1:
      memos[j+1] = subseq
    else:
      memos.append(subseq)
    _ = debug and print(f"{memos=}")
  return memos[-1]

assert ['abzba'] == list(lpsubseq('xaybzba'))
assert ['bab', 'aba'] == list(lpsubseq('abab'))
assert ("aba", "bab") == lcsubstring("abab", "baba")
assert ([3, 2, 1],) == lcsubstring([1, 2, 3, 2, 1], [3, 2, 1, 4, 7])
assert ("abace",) == tuple(lcsubseq("apbcadcqer", "rasbtaucve"))
assert ((9,), (8,), (7,)) == lisubsequence([9, 8, 7])
assert ((1, 5, 6), (1, 2, 3)) == lisubsequence((7, 8, 1, 5, 6, 2, 3))
assert [1, 2, 3] == lisubseq_bottom_up([7, 8, 1, 5, 6, 2, 3], debug=False)
# debug output:
# memos=[[7]]
# memos=[[7], [7, 8]]
# memos=[[1], [7, 8]]
# memos=[[1], [1, 5]]
# memos=[[1], [1, 5], [1, 5, 6]]
# memos=[[1], [1, 2], [1, 5, 6]]
# memos=[[1], [1, 2], [1, 2, 3]]

In [ ]:
def lpsubseq(L):  # LPSubseq, longest palindromic subsequence http://youtu.be/Mbav2iuJ7xQ yapf:disable
  def dp(start, stop):
    if stop - start<=2:  # 1 or 2 chars.
      if L[start] == L[stop-1]:
        yield L[start:stop]
      else:
        yield from (L[start:start+1], L[start+1:stop])
    elif (char := L[start]) == L[stop-1]:
      yield from (char + subseq + char for subseq in dp(start+1, stop-1))
    else:
      yield from (maxima(chain(dp(start+1, stop), dp(start, stop-1)), key=len))

  return dedupe(dp(0, len(L)))

assert ['abzba'] == list(lpsubseq('xaybzba'))
assert ['bab', 'aba'] == list(lpsubseq('abab'))

In [ ]:
def lps(seq):  # LPS, longest palindromic substring.
  n = len(seq)
  def proc():
    for i in range(n):
      for j in range(2):  # cases for even and odd # of chars.
        lo, hi = i, i + j
        while 0 <= lo and hi < n and seq[lo] == seq[hi]:
          lo, hi = lo-1, hi+1
        yield slice(lo+1, hi)

  return seq[max(proc(), key=lambda s: s.stop - s.start)]

assert "bab" == lps("babad")
assert "bb" == lps("cbbd")

In [ ]:
# LC 334: n increasing subsequence https://leetcode.com/problems/increasing-triplet-subsequence/
def find_increasing_subseq_end(iterable, n=3):  # test if it is >-1.
  # subsequence_ends[k] has the end of the best increasing subsequence of length k.
  subseq_ends = []
  for i, q in enumerate(iterable):
    m = len(subseq_ends)
    j = next((j for j, e in enumerate(subseq_ends) if q < e), m)
    subseq_ends[j:j+1] = [q]
    if len(subseq_ends) == n:
      return i
  return -1

seq = [7, 1, 8, 1, 9, 2]
assert 4 == find_increasing_subseq_end(seq, 3)

In [ ]:
def slice_shortest_superset_subarray(L, subset):  # *smallest superset window*
  def prog():
    demand = Counter(subset)
    supply = Counter({k: 0 for k in demand})
    start = 0
    for stop, i in enumerate(L, 1):  # i: in, o: out of a window.
      if i in supply:
        supply[i] += i in supply
        if supply >= demand:
          while supply.get(o := L[start], 1) > demand[o]:  # over-supplied.
            supply[o] -= o in supply
            start += 1
          yield slice(start, stop)

  return L[min(prog(), key=lambda e: e.stop - e.start)]
  # return tuple(L[e] for e in minima(prog(), key=lambda e: e.stop - e.start))

def longest_unique_subarray(L):  # also called the longest distinct slice.
  def prog():
    seen = set()
    start = 0
    for stop, e in enumerate(L):  # e: element being processed.
      if e in seen:
        yield slice(start, stop)
        while e in seen:
          seen.remove(L[start])
          start += 1
      seen.add(e)
    yield slice(start, len(L))

  return tuple(L[s] for s in maxima(prog(), lambda e: e.stop - e.start))

superset_window = ("ab.a", "a.ba")
assert slice_shortest_superset_subarray("ab.a_a.ba", "aba") in superset_window
L = [7, 5, 9, 0, 2, 1, 3, 5, 7, 9, 1, 1, 5, 8, 8, 9, 7]
superset_window = [5, 7, 9, 1], [9, 1, 1, 5]
assert slice_shortest_superset_subarray(L, [5, 1, 9]) in superset_window
assert ("",) == longest_unique_subarray("")
assert ("a",) == longest_unique_subarray("a")
assert ("ab",) == longest_unique_subarray("ab")
assert ("abc", "cde") == longest_unique_subarray("aabccdee")

In [ ]:
def quickfind_median(lhs, rhs):
  if (n := len(lhs) + len(rhs)) % 2:
    return quickfind_n(lhs, rhs, n//2+1)
  else:
    le = quickfind_n(lhs, rhs, n//2)  # less than or equal to the median.
    ge = quickfind_n(lhs, rhs, n//2+1)  # greater than or equal to the median.
    return (le+ge) / 2

def quickfind_n(lhs, rhs, n):  # finds the 1-based n-th smallest integer.
  if len(lhs) > len(rhs):
    return quickfind_n(rhs, lhs, n)
  if not lhs:
    return rhs[n-1]
  if n==1:
    return min(lhs[0], rhs[0])
  l_slice = min(len(lhs), n//2)
  r_slice = n - l_slice
  if lhs[l_slice-1] <= rhs[r_slice-1]:
    return quickfind_n(lhs[l_slice:], rhs, r_slice)
  else:
    return quickfind_n(lhs, rhs[r_slice:], l_slice)

assert 3.5 == quickfind_median([1, 2, 3], [4, 5, 6])
assert 4.5 == quickfind_median([1, 2, 3], [4, 5, 6, 7, 8])
assert 2.0 == quickfind_median([1, 2], [2, 2])

In [ ]:
def reverse_in_n_batch(node, n=2):  # LC 25: Reverse nodes in size-n batches.
  head = tail = None
  while node:
    tail_save, prev, count = node, None, n
    while node and count>0:
      node.next, prev, node, count = prev, node, node.next, count-1
    if tail:
      tail.next = prev
    else:
      head = prev
    tail = tail_save
  return head

seq6 = SNode.from_values(*range(6))
seq5 = SNode.from_values(*range(5))
seq1 = SNode.from_values(*range(1))
assert [1, 0, 3, 2, 5, 4] == [e.data for e in reverse_in_n_batch(seq6, 2)]
assert [2, 1, 0, 4, 3] == [e.data for e in reverse_in_n_batch(seq5, 3)]
assert [0] == [e.data for e in reverse_in_n_batch(seq1, 2)]

In [ ]:
def three_sum(seq, query=0):  # LC 15. 3 sum.
  seq, n = sorted(seq), len(seq)
  for i in range(n-2):
    j, k = i+1, n-1
    while j < k:
      diff = sum((seq[i], seq[j], seq[k])) - query
      if diff<0:
        j += 1
      elif diff>0:
        k -= 1
      else:
        yield (seq[i], seq[j], seq[k])
        j, k = j+1, k-1

def three_sum_closest(seq, q=0):  # LC 16. 3 sum closest.
  seq, n = sorted(seq), len(seq)
  minima = None
  for i in range(n-2):
    j, k = i+1, n-1
    while j < k:
      diff = sum((seq[i], seq[j], seq[k])) - q
      if not minima or abs(diff) < min_diff:
        minima, min_diff = [(seq[i], seq[j], seq[k])], abs(diff)
      elif abs(diff) == min_diff:
        minima.add((seq[i], seq[j], seq[k]))
      if diff<0:
        j += 1
      elif diff>0:
        k -= 1
      else:
        j, k = j+1, k-1
  return minima

assert {(-1, -1, 2), (-1, 0, 1)} == set(three_sum([-1, 0, 1, 2, -1, -4], 0))
assert [(-4, 1, 2)] == three_sum_closest([-1, 2, 1, -4], 0)

In [ ]:
def balance_ab(seq):  # LC 1653: Minimal deletions to balance a's and b's.
  def prog(n):
    if n:
      corrections, bs = prog(n-1)
      if (c := seq[n-1]) == "a":
        cases = ((bs, bs), (corrections+1, bs))  # delete b's or delete one a'.
        return min(cases, key=lambda e: e[0])
      assert c == "b"
      return corrections, bs+1
    return 0, 0

  return prog(len(seq))[0]  # prog returns a tuple (corrections, # of b's).

def balance_ab_bottom_up(seq):
  m = len(seq)
  f = [0] * (m+1)
  b = 0
  for n, c in enumerate(seq, 1):
    if c == "b":
      f[n] = f[n-1]
      b += 1
    else:
      f[n] = min(f[n-1]+1, b)
  return f[m]

assert [2, 3] == [balance_ab(e) for e in ("aababbab", "bbbbaaabb")]
assert [2, 3] == [balance_ab_bottom_up(e) for e in ("aababbab", "bbbbaaabb")]

In [ ]:
def equalize_sums(lesser, greater):  # LC 1775: Minimal swap outs to equalize sums.
  l_sum, g_sum = sum(lesser), sum(greater)
  if l_sum > g_sum:
    return equalize_sums(lesser, lesser)
  correction = [6 - e for e in lesser] + [e-1 for e in greater]
  cum_correction = accumulate(sorted(correction, reversed=True))
  return next((
      i for i, cc in enumerate(cum_correction, 1)  #
      if cc >= g_sum - l_sum), -1)

In [ ]:
def de_equalize_char_frequencies(seq):  # LC 1647: Ensure Char Frequencies Unique.
  it = iter(sorted(Counter(seq).values(), reverse=True))
  corrections, pred = 0, next(it)
  for c in it:
    if pred==0:
      corrections += c
    elif c >= pred:
      corrections += c - pred+1
      pred -= 1
    else:
      pred = c
  return corrections

In [ ]:
def count_4p_family_seats(n_rows, seats_occupied):  # LC 1386. Family Cinema Seats.
  d = defaultdict(int)
  for r, c in seats_occupied:
    d[r] |= 1 << (10 - c)
  masks = (0b0111100000, 0b0000011110, 0b0001111000)
  count = 2 * (n_rows - len(d))  # 2 seats * num empty rows
  for e in d.values():
    for mask in masks:
      if (e & mask)==0:
        e |= mask
        count += 1
  return count

In [ ]:
def dfs(G, v0, entered, exited):  # LC 210: Course Schedule II.
  if v0 not in entered:
    entered.add(v0)
    for v in G.get(v0, {}):
      yield from dfs(G, v, entered, exited)
    exited.add(v0)
    yield EXIT, v0
  elif v0 not in exited:
    raise RuntimeError("Found a cycle at {v0=}")

def topo_sort(G, n=None):
  entered, exited = set(), set()
  n = n or len(G)
  for v0 in range(n):
    if v0 not in entered:
      yield from (v for event, v in dfs(G, v0, entered, exited) if event == EXIT)

def to_graph(prerequisites):
  G = defaultdict(set)
  for a, b in prerequisites:
    G[b].add(a)
  return G

ENTER, CROSS, EXIT = range(1, 4)
prerequisites = [[1, 0], [2, 0], [3, 1], [3, 2]]
assert [3, 1, 2, 0] == list(topo_sort(to_graph(prerequisites)))
assert [0] == list(topo_sort(to_graph([]), 1))

In [ ]:
def append(e):  # LC 295: Track the median of an integer stream.
  heappush(min_q, -heappushpop(max_q, -e))
  if len(min_q) > len(max_q)+1:
    heappush(max_q, -heappop(min_q))

max_q, min_q = [], []  # max_q for negated, smaller half, min_q for others.

In [ ]:
def tic_tac_toe(moves):  # LC 1275: Find a winner on a tic-tac-toe game.
  n = len(moves)
  counts = [0] * 8
  for k in range(n-1, -1, -2):
    r, c = moves[k]
    counts[r] += 1
    counts[3 + c] += 1
    if r == c:
      counts[6] += 1
    if r + c==2:
      counts[7] += 1
  if any(v==3 for v in counts):
    return A if n % 2 else B
  return Pending if n<9 else Draw

Pending, A, B, Draw = range(4)
assert B == tic_tac_toe([[0, 0], [1, 1], [0, 1], [0, 2], [1, 0], [2, 0]])

In [ ]:
def rematch(t, p):  # LC 10: Match a regular expression with "." and "*".
  if not p:  # base cases
    return not t
  if len(p)>1 and p[1] == "*":  # recursive cases
    cases = (t and match1(t, p, ".") and rematch(t[1:], p),
             rematch(t, p[2:]))  # yapf:disable
    return any(cases)
  return t and match1(t, p, ".") and rematch(t[1:], p[1:])

def fnmatch(t, p):  # LC 44: Match a wildcard pattern with "?", and "*".
  if not p:  # base cases
    return not t
  if p[0] == "*":  # recursive cases
    return t and fnmatch(t[1:], p) or fnmatch(t, p[1:])
  return t and match1(t, p, "?") and fnmatch(t[1:], p[1:])

match1 = lambda t, p, any_char: p[0] in {t[0], any_char}
assert rematch("bccdwxpyqqz", "a*b*c*dw.*x.*y.*z")
assert all(fnmatch("xyz", p) for p in ("x*", "x*z", "*z"))

In [ ]:
def bisect_greater(tree, query, succ=None):  # LC 285: Successor in BST.
  while tree:
    if tree.data > query:
      succ = tree
      tree = tree.left
    else:
      tree = tree.right
  return succ

In [ ]:
# 1578 Minimum Time To Make Rope Colorful
def min_time_to_color_rope(colors, costs):
  def prog(m, r_color):
    if m>0:
      color = colors[m-1]
      cases = [prog(m-1, r_color) + costs[m-1]]
      if color != r_color:
        cases.append(prog(m-1, color))
      return min(cases)
    else:
      return 0

  return prog(len(colors), None)

assert 3 == min_time_to_color_rope("abaac", [1, 2, 3, 4, 5])
assert 2 == min_time_to_color_rope("aabaa", [1, 2, 3, 4, 1])

In [ ]:
# Divide a shelf of books into at most k contiguous partitions,
# ensuring the maximum total pages in a partition is minimized.
def bookshelf(books, k):  # at most k partitions.
  book_acc = list(accumulate(books, initial=0))
  book_sum = lambda start, stop: book_acc[stop] - book_acc[start]
  def prog(m, k):  # m books to process, at most k partitions.
    if m>0 and k>0:
      cases = (prog(n, k-1) + ((n, m),) for n in range(m))
      return min(cases, key=lambda e: max(e, key=lambda e: book_sum(e[0], e[1])))
    else:
      return ()

  return prog(len(books), k)

books = [1, 2, 3, 4, 5, 6, 7, 8, 9]
assert ((0, 6), (6, 8), (8, 9)) == bookshelf(books, 3)

In [ ]:
def partition_for_least_sum_diff(L, k):
  def prog(m, *sums):  # m items to process, at most k partitions.
    if m>0:
      n = L[m-1]
      sum_cases = [
          tuple(e + n if i == j else e
                for j, e in enumerate(sums))
          for i, _ in enumerate(sums)
      ]
      sum_cases.append(sums)
      cases = [prog(m-1, *sums) for sums in sum_cases]
      return min(cases, key=lambda case_: max(*case_, grand_sum - sum(case_)))
    else:
      return sums

  grand_sum = sum(L)
  sums = prog(len(L), *[0 for _ in range(k-1)])
  return sums + (grand_sum - sum(sums),)

assert (5, 5, 4, 3) == partition_for_least_sum_diff([3, 1, 2, 2, 4, 5], 4)

# def partition_for_least_sum_diff(L, k):
#   def prog(m, *sums):
#     if m:
#       return min([prog(m-1, *[e + L[m-1] if i == j else e for j, e in enumerate(sums)])
#                   for i, _ in enumerate(sums)] + [prog(m-1, *sums)],
#                  key=lambda case_: max(*case_, sum(L) - sum(case_)))
#     return sums

#   return prog(len(L), *[0] * (k-1)) + (sum(L) - sum(prog(len(L), *[0] * (k-1))),)

# assert (5, 5, 4, 3) == partition_for_least_sum_diff([3, 1, 2, 2, 4, 5], 4)

In [ ]:
def min_meeting_rooms(intervals):  # LC 253: Meeting Rooms II
  heapify(pq := reduce(lambda acc, intvl: acc + [(intvl[0], 1), (intvl[1], -1)],
                       intervals, []))
  return max(accumulate(heappop(pq)[-1] for _ in range(len(pq))))

assert 2 == min_meeting_rooms([[0, 30], [5, 10], [15, 20]])

In [ ]:
# LC 101 symmetric tree?
def symmetric0(tree):  # is a binary tree symmetric?
  if tree is None:
    return True, (), ()  # return (outcome, inorder, and preorder).
  L = symmetric0(tree.left)
  R = symmetric0(tree.right)
  outcome = L[1] == R[1][::-1] and L[2] == R[2]
  in_ = L[1] + (tree.key,) + R[1]
  pre = (tree.key,) + L[2] + R[2]
  return outcome, in_, pre

def symmetric2(L, R):  # symmetric left and right subtrees?
  subtrees = lambda: ((L.left, R.right), (L.right, R.left))
  return (L == R  #
          or (L and R and L.data == R.data  #
              and all(symmetric2(*duo) for duo in subtrees())))

# tree:   1
#       2   2
#     3       3
#      4     4
left = BTree(2, BTree(3, None, BTree(4)))
right = BTree(2, None, BTree(3, BTree(4)))
b7 = BTree(1, left, right)
assert symmetric0(b7)[0]
symmetric = lambda tree: tree and symmetric2(tree.left, tree.right)
assert symmetric(b7)

In [ ]:
# 545. Boundary of Binary Tree (Medium)
# Given a binary tree, find its side of left side nodes, leaf, and right side nodes.
def is_leaf(tree):
  return not any((tree.left, tree.right))

def dfs_left(tree, side):
  if tree:
    if side or is_leaf(tree):
      yield tree.data
    yield from dfs_left(tree.left, side)
    yield from dfs_right(tree.right, False)

def dfs_right(tree, side):
  if tree:
    if side or is_leaf(tree):
      yield tree.data
    yield from dfs_right(tree.right, side)
    yield from dfs_left(tree.left, False)

def dfs_exterior(tree):
  if tree:
    yield tree.data
    yield from dfs_left(tree.left, True)
    yield from reversed(list(dfs_right(tree.right, True)))

# tree:     a
#         /   \
#       b      f
#     c  e   g  h
#   d       i

d = BTree('d')
c = BTree('c', d)
e = BTree('e')
b = BTree('b', c, e)
tree_a = BTree('a', b, BTree('f', BTree('g', BTree('i')), BTree('h')))
assert "abcdeihf" == "".join(dfs_exterior(tree_a))

In [ ]:
def acceptable(candidate, j):
  i = candidate[-1]
  k = i ^ j
  return k and (k & k-1) == 0

def gray_code(n, candidate=None, accepted=None):
  candidate = candidate or []
  accepted = accepted or set()
  if len(candidate) == 2**n:
    if acceptable(candidate, candidate[0]):
      yield candidate[:]
  else:
    for e in range(2**n):
      if not accepted or e not in accepted and acceptable(candidate, e):
        candidate.append(e)
        accepted.add(e)
        yield from gray_code(n, candidate, accepted)
        accepted.remove(e)
        candidate.pop()

def grey_code(n):
  if n == 1:
    yield from ((0, 1), (1, 0))
  for e in grey_code(n-1):
    yield e
  for e in reversed(grey_code(n-1)):
    yield e

assert [0, 1, 3, 2] == next(gc := gray_code(2), None)
assert 7 == len(tuple(gc))

In [ ]:
# 2035 Partition Array Into Two Arrays to Minimize Sum Difference
recap = lambda kv, k, v: tuple(p + q for p, q in zip(kv, (k, v)))

def partition_for_least_sum_diff(seq):
  def prog(n, c):  # n elements to partition, c: capacity.
    if n:
      e = seq[n-1]
      cases = (
          recap(prog(n-1, c - e), 0, indices)  #
          for e, indices in ((e, (n-1,)), (0, ()))
          if c >= e)
      return min(cases, key=lambda kv: kv[0])
    else:
      return (c, ())

  return prog(len(seq), sum(seq)//2)

seq = [3, 9, 7, 3]
partition_for_least_sum_diff([2, -1, 0, 4, -2, -9])

In [ ]:
def max_subseq_union_length(seq):
  bits_all = [0]
  for s in seq:
    bits = 0
    for i in map(lambda c: ord(c) - ord("A"), s):
      bit = 1 << i
      if bits & bit:
        break
      bits |= bit
    else:
      bits_all.extend((e | bits) for e in bits_all if (e & bits)==0)
  return max(e.bit_count() for e in bits_all)

max_subseq_union_length(["cha", "r", "act", "ers"])

In [ ]:
def longest_path(parents, values):
  def dfs(tree: int):
    c_dfs = [(c, dfs(c)) for c in g[tree]]
    lens = list(map(lambda e: e[1], c_dfs))
    len_ = 1 + sum(l for (c, l) in c_dfs if values[tree] != values[c])
    return max(lens + [len_])

  g = defaultdict(list)
  for c, e in enumerate(parents):
    g[e].append(c)
  return dfs(0)

parents = [-1, 0, 0, 1, 1, 2]
longest_path(parents, "abacbe")

In [ ]:
def maximalNetworkRank(n, roads) -> int:
  g = [[0] * n for _ in range(n)]
  cnt = [0] * n
  for a, b in roads:
    g[a][b] = g[b][a] = 1
    cnt[a] += 1
    cnt[b] += 1
  return max(cnt[a] + cnt[b] - g[a][b] for a in range(n) for b in range(a+1, n))

In [ ]:
def count_islands(grid):
  def dfs(r0, c0):
    grid[r0][c0] = "0"
    for rd, cd in pairwise(dirs):
      r, c = r0 + rd, c0 + cd
      if 0 <= r < m and 0 <= c < n and grid[r][c] == "1":
        dfs(r, c)

  count = 0
  dirs = (-1, 0, 1, 0, -1)
  m, n = len(grid), len(grid[0])
  for i in range(m):
    for j in range(n):
      if grid[i][j] == "1":
        dfs(i, j)
        count += 1
  return count

In [ ]:
def spiral_iterate(matrix):
  m, n = len(matrix), len(matrix[0])
  dirs = (0, 1, 0, -1, 0)
  seen = defaultdict(bool)
  r = c = d = 0
  for _ in range(m * n):
    yield matrix[r][c]
    seen[r, c] = True
    r2, c2 = r + dirs[d], c + dirs[d+1]
    if not (0 <= r2 < m and 0 <= c2 < n) or seen[r2, c2]:
      d = (d+1) % 4
    r, c = r + dirs[d], c + dirs[d+1]

tuple(spiral_iterate([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]))

In [ ]:
def iter_diagonal(grid):
  m, n = len(grid), len(grid[0])
  dirs = (-1, 1), (1, -1),
  def offset(k):
    if k % 2:
      return 0 if k < n else k - n+1, k if k < n else n-1
    else:
      return k if k < m else m-1, 0 if k < m else k - m+1
  def iterate_once(k):
    r, c = offset(k)
    while 0 <= r < m and 0 <= c < n:
      yield grid[r][c]
      r, c = (e + d for e, d in zip((r, c), dirs[k % 2]))
  def iterate_all():
    yield from (iterate_once(k) for k in range(m + n-1))

  yield from (e for once in iterate_all() for e in once)

expected = (1, 2, 5, 9, 6, 3, 4, 7, 0, 1, 8, 2)
assert expected == tuple(
    iter_diagonal([  #
        [1, 2, 3, 4],  #
        [5, 6, 7, 8],  #
        [9, 0, 1, 2]
    ]))

In [ ]:
# # 768 Max Chunks To Make Sorted II
# def max_chunks(seq):
#   cummax =
#   cummin =
#   slice_non_sorted = +1

In [ ]:
class Trie:
  def __init__(self, value=None, **_):
    self.value, self.children = value, defaultdict(lambda: Trie())
  def __setitem__(self, key, child):
    if len(key)>1:
      self.children[key[0]][key[1:]] = child
    else:
      self.children[key[0]] = child if isinstance(child, Trie) else Trie(child)
  def __getitem__(self, key):
    return (c := self.children.get(key[0], None)) and c[key[1:]] if key else self
  def values(self):
    if self.value is not None:
      yield self.value
    for child in self.children.values():
      yield from child.values()

def boggle_game_dfs(game):
  def within_bounds_n_limits(r, c, candidate):
    return -1 < r < m and -1 < c < n and (r, c) not in candidate
  def expand_out(candidate):
    (r0, c0), tree0 = next(reversed(candidate.items()))
    for r_, c_ in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0),
                   (1, 1)]:
      r, c = r0 + r_, c0 + c_
      if within_bounds_n_limits(r, c, candidate) and (tree := tree0[game[r][c]]):
        yield (r, c), tree

  m, n = len(game), len(game[0])
  def backtrack(candidate):
    (r0, c0), tree0 = next(reversed(candidate.items()))
    if tree0.value is not None:
      yield vocabs[tree0.value]
    else:
      for (r, c), tree in expand_out(candidate):
        candidate[(r, c)] = tree
        yield from backtrack(candidate)
        candidate.popitem()

  for r in range(m):
    for c in range(n):
      if (tree := trie[game[r][c]]):
        yield from backtrack({(r, c): tree})

trie = Trie()
for i, e in enumerate(vocabs := ("GEEKS", "FOR", "QUIZ", "GO")):
  trie[e] = i
game = [['G', 'I', 'Z'], ['U', 'E', 'K'], ['Q', 'S', 'E']]
assert ("GEEKS", "QUIZ") == tuple(boggle_game_dfs(game))

In [ ]:
def word_ladder_puzzle_bfs(source, sink):
  q = deque([(source,)])
  seen = set()
  while q:
    ladder = q.popleft()
    if (word0 := ladder[-1]) == sink:
      yield ladder
    elif word0 not in seen:
      seen.add(word0)
      for i in range(len(word0)):
        for c in string.ascii_lowercase:
          if (word := f"{word0[:i]}{c}{word0[i+1:]}") in vocabs:
            q.append(ladder + (word,))

vocabs = ("hot", "dot", "dog", "lot", "log", "cog")
word_ladder = "hit", "hot", "dot", "dog", "cog"
assert word_ladder == next(word_ladder_puzzle_bfs("hit", "cog"))

In [ ]:
def dfs(graph, v0, entered=None, exited=None):  # initial_edge, also called start_edge.
  if v0 not in entered:
    entered.add(v0)
    for v in graph.get(v0, {}):
      yield from dfs(graph, v, entered, exited)
    exited.add(v0)
    yield EXIT, v0
  elif v0 not in exited:
    raise RuntimeError(f"Cycle detected at {v0}")

def topological_sort(graph):
  entered, exited = set(), set()
  for v in graph:
    yield from (w for op, w in dfs(graph, v, entered, exited) if op == EXIT)

def alien(vocabs):
  try:
    return "".join(topological_sort(to_graph(vocabs)))
  except RuntimeError:
    ...

def to_graph(vocabs):
  w_edges = ((w, v)  # word w has a higher rank than word v
             for (i, v), (j, w) in product(enumerate(vocabs), repeat=2)
             if i < j)
  c_edges = ((b, a)  # alphabet b has a higher rank than alphabet a.
             for w, v in w_edges
             for i, (b, a) in enumerate(zip(w, v))
             if w[:i] == v[:i] and b != a)
  return reduce(lambda G, edge: G[edge[0]].add(edge[1]) or G, c_edges,
                defaultdict(set))

ENTER, CROSS, EXIT = range(1, 4)
assert "wertf" == alien(["wrt", "wrf", "er", "ett", "rftt"])
assert alien(["w", "x", "w"]) is None

In [ ]:
def bus_route_bfs(routes: list[list[int]], source: int, sink: int):
  g = defaultdict(list)
  for i_route, route in enumerate(routes):
    for stop in route:
      g[stop].append(i_route)
  q = [(source, rides := (source,))]
  routes_seen = set()
  for stop0, rides in q:
    if stop0 == sink:
      yield rides
    for i_route in g[stop0]:
      if i_route not in routes_seen:
        routes_seen.add(i_route)
        for stop in routes[i_route]:
          q.append((stop, rides + (stop,)))

routes3 = [[1, 2, 3], [2, 4, 5], [5, 6, 7], [3, 8, 9], [9, 10, 11]]
assert (1, 3, 9, 11) == next(bus_route_bfs(routes3, 1, 11))
assert (1, 3) == next(bus_route_bfs([[1, 2, 3], [4, 5, 6]], 1, 3))
assert () == tuple(bus_route_bfs([[1, 2, 3], [4, 5, 6]], 1, 6))

In [ ]:
def slide_puzzle_bfs(game):
  def expand_out(i0):
    r0, c0 = i0//n, i0%n  # yapf:disable
    for r, c in ((r0, c0-1), (r0, c0+1), (r0-1, c0), (r0+1, c0)):
      if -1 < r < m and -1 < c < n:
        yield r*n + c

  done = lambda pickled: all(i == e for i, e in enumerate(pickled, 1) if e)
  m, n = len(game), len(game[0])
  G = list(chain(*game))
  q = deque([(G, (G.index(0),))])
  seen = set()
  while q:
    G0, moves0 = q.popleft()
    if done(G0):
      yield moves0
    elif tuple(G0) not in seen:
      seen.add(tuple(G0))
      i0 = G0.index(0)
      for i in expand_out(i0):
        G = G0[:]
        G[i0], G[i] = G[i], G[i0]
        q.append((G, moves0 + (i,)))

game0 = [[1, 2, 3], [5, 0, 4]]
game1 = [[1, 2, 3], [4, 0, 5]]
game2 = [[0, 2, 3], [5, 1, 4]]
assert [] == list(slide_puzzle_bfs(game0))
assert (4, 5) == next(slide_puzzle_bfs(game1))
assert (0, 3, 4, 5, 2, 1, 0, 3, 4, 1, 2, 5) == next(slide_puzzle_bfs(game2))

In [ ]:
def sum_rainwater(E):  # L: elevations, also called heights.
  l_cummax = accumulate(E, max)  # O(1) space.
  r_cummax = reversed(list(accumulate(reversed(E), max)))  # O(n) space.
  return sum([min(l, r) - e for (l, r, e) in zip(l_cummax, r_cummax, E)])

assert 6 == sum_rainwater([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1])
assert 9 == sum_rainwater([4, 2, 0, 3, 2, 5])

In [ ]:
def dijkstra_n(G, source, sink, n=1):  # dijkstra in max n-stop flights.
  seen = set()
  costs = {(source, -1): 0}
  heap = [(0, source, -1)]  # at the source with cost(0) at stop count(-1).
  while heap:
    cost0, v0, n_stops = heappop(heap)
    if (v0, n_stops) not in seen:
      seen.add((v0, n_stops))
      if v0 == sink:
        return cost0
      if (n_stops := n_stops+1) <= n:
        for v, c in G[v0].items():
          if (cost := cost0 + c) < costs.get((v0, n_stops), INF):
            heappush(heap, (cost, v, n_stops))
            costs[(v, n_stops)] = cost
  return -1

def to_graph(flights):
  G = defaultdict(dict)
  for from_, to, cost in flights:
    G[from_][to] = cost
  return G

search_flights = lambda flights, s, d, n: dijkstra_n(to_graph(flights), s, d, n)
flights = [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]]
assert 700 == search_flights(flights, 0, 3, 1)
assert 400 == search_flights(flights, 0, 3, 2)
assert -1 == search_flights(flights, 1, 0, 0)
assert 500 == search_flights([[0, 1, 100], [1, 2, 100], [0, 2, 500]], 0, 2, 0)
assert 200 == search_flights([[0, 1, 100], [1, 2, 100], [0, 2, 500]], 0, 2, 1)

In [ ]:
def pour_water(heights, volume: int, k: int) -> int:
  for _ in range(volume):
    for d in (-1, 1):
      i = j = k
      while 0 <= i + d < len(heights) and heights[i + d] <= heights[i]:
        if heights[i + d] < heights[i]:
          j = i + d
        i += d
      if j != k:
        heights[j] += 1
        break
    else:
      heights[k] += 1
  return heights

In [ ]:
def guess_secret(words, master):
  for i in range(10):
    guess = random.choice(words)
    x = max(0, master.guess(guess))
    words = [w for w in words if sum(i == j for i, j in zip(guess, w)) == x]

In [ ]:
def make_changes(denoms, k):  # Make the changes k with least coins.
  def prog(n, k):  # n denominations.
    d = denoms[n-1]
    if n>1:
      yield from prog(n-1, k)
      if k >= d:
        yield from (e + (d,) for e in prog(n, k - d))
    elif k % d==0:
      yield (d,) * (k//d)

  return min(prog(len(denoms), k), default=None, key=lambda v: len(v))

def make_amount(denoms, k):  # Make amount with denominations[i].
  def prog(n, k):  # n denominations.
    d = denoms[n-1]
    if n>1:
      yield from prog(n-1, k)
      if k >= d:
        yield from (e + (d,) for e in prog(n, k - d))
    elif k % d==0:
      yield (d,) * (k//d)

  yield from prog(len(denoms), k)

denominations = [1, 5, 10, 25]
assert 1 == ilen(make_amount(denominations, 4))
assert 2 == ilen(make_amount(denominations, 5))
assert 4 == ilen(make_amount(denominations, 10))  # 3 ways to make 10 cents.
assert 9 == ilen(make_amount(denominations, 20))  # 9 ways to make 20 cents.
assert (5, 5) == make_changes([7, 5, 1], 10)
assert (7, 5, 1) == make_changes([7, 5, 1], 13)
assert (7, 7) == make_changes([7, 5, 1], 14)
assert () == make_changes([7, 5, 1], 0)
assert make_changes([7, 5], 11) is None

In [ ]:
def to_cidr(ip, n):
  to_int = lambda ip: sum(int(e) << w for w, e in zip((24, 16, 8, 0), ip.split(".")))
  to_ip = lambda n: ".".join(str((n >> w) % 256) for w in (24, 16, 8, 0))
  base = to_int(ip)
  while n:
    mask = max(33 - (base & ~(base-1)).bit_length(), 33 - n.bit_length())
    yield f"{to_ip(base)}/{mask}"
    base += 1 << (32 - mask)
    n -= 1 << (32 - mask)

expected = "255.0.0.7/32", "255.0.0.8/29", "255.0.0.16/32"
assert expected == tuple(to_cidr("255.0.0.7", 10))

In [ ]:
def missing_positive(seq):
  n = len(seq)
  for i in range(n):
    while 0 < seq[i] <= n and seq[i] != seq[seq[i]-1]:
      tmp = seq[i]
      seq[i], seq[tmp-1] = seq[tmp-1], tmp
  for i in range(1, n+1):
    if i != seq[i-1]:
      return i
  return n+1

assert 4 == missing_positive([1, 2, 3])
assert 2 == missing_positive([3, -1, 5, 1])

In [ ]:
def max_area(heights):  # LC 11: a container with most water.
  def proc():
    start, stop = 0, len(heights)-1
    while start < stop:
      yield (stop-start) * min(heights[start], heights[stop])
      if heights[start] < heights[stop]:
        start += 1
      else:
        stop -= 1

  return max(proc())

assert 49 == max_area([1, 8, 6, 2, 5, 4, 8, 3, 7])
assert 1 == max_area([1, 1])

In [ ]:
# https://leetcode.com/problems/shortest-path-to-get-all-keys/
def find_keys_bfs(map):  # finds the shortest paths to all keys.
  def within_bounds_n_limits(r, c, keys):
    return -1 < r < m and -1 < c < n and (map[r][c].islower() or map[r][c] in keys)
  def expand_out(r0, c0, keys0):
    for r, c in [(r0+1, c0), (r0, c0+1), (r0-1, c0), (r0, c0-1)]:
      if within_bounds_n_limits(r, c, keys0):
        keys = keys0 | {e.upper()} if (e := map[r][c]).islower() else keys0
        yield r, c, keys

  m, n = len(map), len(map[0])
  r, c = next((r, c) for r in range(m) for c in range(n) if map[r][c] == "@")
  q = deque([(route := [(r, c)], keys := frozenset(["@", "."]))])
  key_count = 2 + len({e for row in map for e in row if e.islower()})
  seen = set()
  while q:
    route0, keys0 = q.popleft()
    if len(keys0) == key_count:
      yield route0[1:]
    elif (route0[-1], keys) not in seen:  # prune the same (r, c), keys.
      seen.add((route0[-1], keys))
      for r, c, keys in expand_out(*route0[-1], keys0):
        route = route0 + [(r, c)]
        q.append((route, keys))

map0 = ["@Aa"]
map6 = [
    "@..aA",
    "..B#.",
    "....b"
]  # yapf:disable
map8 = [
    "@.a..",
    "###.#",
    "b.A.B"
]  # yapf:disable
route6 = [(0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4)]
route8 = [(0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 2), (2, 1), (2, 0)]

assert () == tuple(find_keys_bfs(map0))
assert route6 == next(find_keys_bfs(map6))
assert route8 == next(find_keys_bfs(map8))

In [ ]:
# https://github.com/doocs/leetcode/blob/main/solution/0800-0899/0829.Consecutive%20Numbers%20Sum/README_EN.md
def count_consec_int_partitions(n: int) -> int:
  n *= 2
  k = 1
  cases = 0
  while k * (k+1) <= n:
    if n % k==0 and (n//k - k+1) % 2==0:
      cases += 1
    k += 1
  return cases

In [ ]:
def wiggle_sort(nums: list[int]) -> None:
  arr = sorted(nums)
  n = len(arr)
  i, j = (n-1)>>1, n-1
  for k in range(n):
    if k % 2==0:
      nums[k] = arr[i]
      i -= 1
    else:
      nums[k] = arr[j]
      j -= 1

wiggle_sort(nums := [1, 5, 1, 1, 6, 4])
assert [1, 6, 1, 5, 1, 4] == nums

In [ ]:
def smallest_common_region(region1, region2):
  seen = set()
  while parents.get(region1):
    seen.add(region1)
    region1 = parents[region1]
  while parents.get(region2):
    if region2 in seen:
      return region2
    region2 = parents[region2]

regions = [  #
    ["Earth", "North America", "South America"],
    ["North America", "United States", "Canada"],
    ["United States", "New York", "Boston"], ["Canada", "Ontario", "Quebec"],
    ["South America", "Brazil"]
]
parents = {r: region[0] for region in regions for r in region[1:]}
assert "North America" == smallest_common_region("Quebec", "New York")

In [ ]:
def paginate_airbnb(size: int, iterable):  # at most 1 listing per host in a page.
  def iterate_page():
    top_n = list(heappop(q)[1] for _ in q)
    for host_id in top_n:
      yield listings[host_id].popleft()
      if listings[host_id]:
        heappush(q, (-listings[host_id][0].score, host_id))

  listings = defaultdict(deque)
  q = []
  for e in iterable:
    if len(q) < size:
      if not listings[e.host_id]:
        heappush(q, (-e.score, e.host_id))
      listings[e.host_id].append(e)
    else:
      yield from iterate_page()
  while q:
    yield from iterate_page()

source = [
    "1,28,300.1,San Francisco", "4,5,209.1,San Francisco", "20,7,208.1,San Francisco",
    "23,8,207.1,San Francisco", "16,10,206.1,Oakland", "1,16,205.1,San Francisco",
    "1,31,204.6,San Francisco", "6,29,204.1,San Francisco", "7,20,203.1,San Francisco",
    "8,21,202.1,San Francisco", "2,18,201.1,San Francisco", "2,30,200.1,San Francisco",
    "15,27,109.1,Oakland", "10,13,108.1,Oakland", "11,26,107.1,Oakland",
    "12,9,106.1,Oakland", "13,1,105.1,Oakland", "22,17,104.1,Oakland",
    "1,2,103.1,Oakland", "28,24,102.1,Oakland", "18,14,11.1,San Jose",
    "6,25,10.1,Oakland", "19,15,9.1,San Jose", "3,19,8.1,San Jose", "3,11,7.1,Oakland",
    "27,12,6.1,Oakland", "1,3,5.1,Oakland", "25,4,4.1,San Jose", "5,6,3.1,San Jose",
    "29,22,2.1,San Jose", "30,23,1.1,San Jose"
]
Listing = namedtuple("Listing", "host_id listing_id score city")
Listing.from_csv = classmethod(lambda cls, csv: (v := csv.split(",")) and \
                               cls(int(v[0]), int(v[1]), float(v[2]), v[3]))
source = list(map(Listing.from_csv, source))
expected = (28, 5, 7, 8, 10, 16, 31, 29, 21, 18, 27, 13, 9)
assert expected == tuple(e.listing_id for e in iterate_pages(6, source))[:13]

In [ ]:
def flood_fill_distance(grid):
  gates = [(r, c) for r, row in enumerate(grid) for c, e in enumerate(row) if e==0]
  q = deque(gates)
  def within_bounds_n_limits(r, c):
    return 0 <= r < len(grid) and 0 <= c < len(grid[0]) and grid[r][c] == INF

  while q:
    r0, c0 = q.popleft()
    for r, c in ((r0+1, c0), (r0-1, c0), (r0, c0+1), (r0, c0-1)):
      if within_bounds_n_limits(r, c):
        grid[r][c] = grid[r0][c0]+1
        q.append((r, c))
  return grid

from sys import maxsize as INF
parking_lot = [
    [INF,  -1,   0, INF],
    [INF, INF, INF,  -1],
    [INF,  -1, INF,  -1],
    [  0,  -1, INF, INF],
]  # yapf:disable
distances = [
    [3, -1, 0,  1],
    [2,  2, 1, -1],
    [1, -1, 2, -1],
    [0, -1, 3,  4]
]  # yapf:disable
assert distances == flood_fill_distance(parking_lot)

In [ ]:
# https://leetcode.com/problems/increasing-triplet-subsequence/description/
def multiply_matrices(dense_a, dense_b):
  def sparsify(m):
    return {(r, c): e for r, row in enumerate(m) for c, e in enumerate(row) if e!=0}

  if len(dense_a[0]) == len(dense_b):
    sparse_a, sparse_b = sparsify(dense_a), sparsify(dense_b)
    matrix_c = [[0 for _ in range(len(dense_b))] for _ in range(len(dense_a))]
    for (i, k), e in sparse_a.items():
      for j in range(len(dense_b[0])):
        matrix_c[i][j] += e * sparse_b.get((k, j), 0)
    return matrix_c

In [ ]:
# Find the celebrity https://leetcode.ca/all/277.html
def find_celebrity(matrix) -> int:
  candidate = 0
  for i in range(1, n):
    if matrix[candidate, i]:
      candidate = i

  for i in range(n):
    if i != candidate and (matrix[candidate, i] or not matrix[i, candidate]):
      return -1

  return candidate

In [ ]:
def viterate_bfs(tree):  # LC 987: Vertical order traversal
  v = defaultdict(list)  # trees by column c.
  q = deque([(0, tree)])
  while q:
    c, e = q.popleft()
    if e:
      v[c].append(e)
      q.append((c-1, e.left))
      q.append((c+1, e.right))
  yield from (e for c, l in sorted(v.items()) for e in l)

def zigzag_iterate_bfs(tree):  # LC 103: Zigzag Breadth-first traversal.
  q, d = deque([tree]), 0
  while q:
    for _ in range(len(q)):
      yield (tree := q.pop() if d % 2 else q.popleft())
      for c in (tree.right, tree.left) if d % 2 else (tree.left, tree.right):
        if c:
          q.appendleft(c) if d % 2 else q.append(c)
    d += 1

#      3
#    /   \
#   1     5
#  0 2   4 6
#           7
t8 = btree_from_values(BTree, range(8))
#     2
#   /   \
#  0     4
# x 1   3 5
t6 = btree_from_values(BTree, range(6))
data = lambda iterable: tuple(e.data for e in iterable)
assert (0, 1, 3, 2, 4, 5, 6, 7) == data(viterate_bfs(t8))
assert (0, 2, 1, 3, 4, 5) == data(viterate_bfs(t6))
assert (2, 4, 0, 1, 3, 5) == data(zigzag_iterate_bfs(t6))

In [ ]:
def generate_pyramid_dfs(base_layer: str, allowed: list[str]) -> Iterable[str]:
  def dfs(base):
    if len(base)>1:
      block_choices = (rules[l, r] for l, r in pairwise(base))
      for layer in tuple(map("".join, product(*block_choices))):
        for pyramid in dfs(layer):
          yield f"{base},{pyramid}"
    else:
      yield base

  rules = defaultdict(list)
  for a, b, c in allowed:
    rules[a, b].append(c)
  yield from dfs(base_layer)

assert "BCD,CE,A" == next(generate_pyramid_dfs("BCD", ["BCC", "CDE", "CEA", "FFF"]))

In [ ]:
# Given a nested list of integers, write a program that calculates the depth-weighted sum of all integers in the list.
def proc_bfs(seq):
  q = deque([(seq, d := 0)])
  while q:
    x, d = q.popleft()
    if isinstance(x, (list, tuple)):
      for e in x:
        q.append((e, d+1))
    else:
      yield d * x

def proc_dfs(seq):
  s = [(seq, d := 1, i := 0)]
  while s:
    seq, d, i = s.pop()
    while i < len(seq):
      if isinstance(seq[i], (list, tuple)):
        s.append((seq, d, i+1))
        seq, d, i = seq[i], d+1, 0
      else:
        yield d * seq[i]
        i += 1

def iter_dfs(seq, d=1):
  for e in seq:
    if isinstance(e, (list, tuple)):
      yield from iter_dfs(e, d+1)
    else:
      yield d * e

weighted_sum_bfs = lambda seq: sum(proc_bfs(seq))
weighted_sum_dfs = lambda seq: sum(proc_dfs(seq))
weighted_sum_itr = lambda seq: sum(iter_dfs(seq))
s = [1, [2, [3], 2], 1]
assert (19, 19, 19) == (weighted_sum_bfs(s), weighted_sum_dfs(s), weighted_sum_itr(s))

In [ ]:
def decode_string(seq):  # LC 394. Decode String, "3[a2[c]]" to "accaccacc".
  q, n = deque(), 0
  stack = []
  for c in seq:
    if c.isdigit():
      n = n*10 + int(c)
    elif c.isalpha():
      q.append(c)
    elif c == "[":
      stack.append((q, n))
      q, n = deque(), 0
    elif c == "]":
      q0, n0 = stack.pop()
      q0.append(n0 * "".join(q))
      q = q0
  return "".join(q)

assert "abcccbcccabcccbcccabcccbccc" == decode_string("3[a2[b3[c]]]")

In [ ]:
# Find the optimal buying and selling strategy for k stock trades over n days.
def maximize_profit_of_stock_trades(L, k):  # at most k trades.
  def prog(d, k):  # d day stock prices, at most k trades.
    recap = lambda kv, k, v: (kv[0] + k, kv[1] + v)
    cases = (
        recap(prog(b, k-1), gain, ((b, s),))
        for b in range(0, d-1)
        for s in range(b+1, d)
        if k>0 and (gain := L[s] - L[b])>0)
    return max(cases, default=(0, ()))

  return prog(len(L), k)

L = [10, 5, 15, 20, 12, 18, 25, 10, 15]
assert (33, ((1, 3), (4, 6), (7, 8))) == maximize_profit_of_stock_trades(L, 3)
assert (0, ()) == maximize_profit_of_stock_trades([2, 1], 3)

In [ ]:
def knapsack01(skus, capacity):  # a seq of costs & values.
  @lru_cache(maxsize=None)  # by capacity c and n items.
  def prog(n, c):  # returns a tuple of (a yield, items).
    if n==0:
      return (0, ())  # a tuple of ($0, none items).
    sku = skus[n-1]
    cases = (recap(prog(n-1, c - cost), value, seq)
             for (cost, value), seq in ((sku, (n-1,)), ((0, 0), ()))
             if c >= cost)  # at most two cases
    return max(cases, key=lambda kv: kv[0])

  return prog(len(skus), capacity)

def knapsack_unbounded(skus, capacity):
  @lru_cache(maxsize=None)
  def prog(c):
    cases = (recap(prog(c - sku[0]), sku[1], (i,))
             for i, sku in enumerate(skus)
             if c >= sku[0])
    return max(cases, key=lambda kv: kv[0], default=(0, ()))

  return prog(capacity)

skus = [(2, 2), (1, 1), (4, 10), (1, 2), (12, 4)]
assert (15, (0, 1, 2, 3)) == knapsack01(skus, 16)
assert (36, (3, 3, 3, 2, 2, 2)) == knapsack_unbounded(skus, 15)

In [21]:
def clone(tree):  # make a copy of a binary tree with _x.
  if not tree:
    return
  queue = deque([(tree, tree_out := BTree(tree.data))])
  x = {}
  while queue:
    tree, copy = queue.popleft()
    if tree.left:
      copy.left = BTree(tree.left.data)
      queue.append((tree.left, copy.left))
    if tree.right:
      copy.right = BTree(tree.left.data)
      queue.append((tree.right, copy.right))
    if tree in x:
      x.pop(tree)._x = tree
    if tree._x:
      x[tree._x] = copy
  return tree_out